In [2]:
import os
import pandas
import requests
from newspaper import Article, Config
from bs4 import BeautifulSoup
from multiprocessing import Pool


length = 50
total_length = 500

group_items = {}

url = 'https://www.cupoy.com/MixNewsMongoAction.do'

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [1]:
def write_article_content(article_id, url, description=None):

    # Need Headers user agent: https://www.newmobilelife.com
    config = Config()
    config.browser_user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    article = Article(url, config=config)
    article.download()
    article.parse()
    
    if not article.text:
        response = requests.get(url).text
        soup = BeautifulSoup(response)

        if soup.find('article'):
            # www.bnext.com.tw
            text = [p.text for p in soup.find('article').find_all('p')]
            article.text = '\n'.join(text)
        elif 'www.jiqizhixin.com' in url:
            text = [p.text for p in soup.find_all('section')]
            article.text = '\n'.join(text)
        
        if description and not article.text:
            article.text = description
        
        # selinum https://www.managertoday.com.tw/


    save_dir = 'articles'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    article_path = os.path.join(save_dir, '{:04d}.txt'.format(article_id))
    with open(article_path, 'w') as f_:
        f_.write(article.title + '\n')
        f_.write(article.text)

In [3]:
def get_articles(start_news_id, length):
    
    items = pandas.DataFrame()
    payload = {'op': 'getTopMixNews', 'nationid': 'TW', 'startNewsID': start_news_id, 'len': length}

    try:
        count = 0
        response = requests.get(url, params=payload).json()
        for item in response['items']:
            article_id = start_news_id + count
            item_df = pandas.DataFrame({
                'title': item.get('title'),
                'description': item.get('description'),
                'linkurl': item.get('linkurl'),
                'keywordtrend': item['metrics']['keywordtrend'],
                'uidactscore': item['metrics']['uidactscore'],
                'socialactscore': item['metrics']['socialactscore'],
                'activescore': item['metrics']['activescore'],
                'groups': (str(item['bucketgrpids']) if 'bucketgrpids' in item else None)
            }, index=[article_id])

            print(article_id, item['title'], sep='\t')
            if item.get('linkurl'):
                write_article_content(article_id, item['linkurl'], item['description'])
            
            items = items.append(item_df)
            count +=1
        return items
    except Exception as err:
        print(err)
        return



In [4]:
items = pandas.DataFrame()
with Pool() as pool:

    results = [pool.apply_async(get_articles, (start_news_id, length)) for start_news_id in range(0, total_length, length)]
    
    for result in results:
        items = items.append(result.get())




150	〈定存理財術〉儲蓄前先做三件事 一年至少多領8753元起跳-鉅亨網-觀點新聞-商周財富網
50	CMHK 推抗疫八招：免費送 5GB 全速數據，免數據睇 UTV (包括有線新聞臺)！ | Qooah
100	用復古未來主義設計電話！美國傳奇工業設計師用未來感完美復刻古董電話Bell 302-風傳媒
Building prefix dict from /home/susan8213/.pyenv/versions/venv-pycrawler-cupoy/lib/python3.6/site-packages/jieba/dict.txt ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from /home/susan8213/.pyenv/versions/venv-pycrawler-cupoy/lib/python3.6/site-packages/jieba/dict.txt ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from /home/susan8213/.pyenv/versions/venv-pycrawler-cupoy/lib/python3.6/site-packages/jieba/dict.txt ...
Loading model from cache /tmp/jieba.cache
0	疫情帶動宅經濟，電商徵才月薪上看 10 萬元 | TechNews 科技新報
Building prefix dict from /home/susan8213/.pyenv/versions/venv-pycrawler-cupoy/lib/python3.6/site-packages/jieba/dict.txt ...
Loading model from cache /tmp/jieba.cache
Loading model cost 2.8760409355163574 seconds.
Prefix dict has been built succesfully.
Loading model cost 2.9840006828308105 seconds.

In [5]:
items.to_csv('topnews.csv')
items

,title,description,linkurl,keywordtrend,uidactscore,socialactscore,activescore,groups
0,疫情帶動宅經濟，電商徵才月薪上看 10 萬元 | TechNews 科技新報,武漢肺炎疫情升溫，百貨業績下滑，以往熱鬧的臺北市信義區連週末都人潮稀少，反觀網購商機逆勢爆發...,https://technews.tw/2020/02/15/wuhan-pneumonia...,0,85,0,29988,"['3CExpert_tw', 'business_tw', 'tech_tw']"
1,臺積電先進製程產能太搶手，華為受肺炎疫情衝擊也不敢砍單 | TechNews 科技新報,在中國武漢肺炎疫情的衝擊下，之前有外電引用分析機構的研究數據顯示，中國手機市場的出貨量將會因...,https://technews.tw/2020/02/15/tsmc-vs-huawei/...,0,85,0,29988,"['3CExpert_tw', 'tech_tw']"
2,退出主場！三星不在南韓販售 Exynos 處理器 S20 手機 | TechNews 科技新報,南韓手機大廠三星電子（Samsung Electronics）最新旗艦機 Galaxy S2...,https://technews.tw/2020/02/14/samsung-do-not-...,0,85,0,29988,"['3CExpert_tw', 'tech_tw']"
3,駭客、工程師鍵盤救國！實名系統 72 小時上線幕後 | TechNews 科技新報,臺灣，正上演一場獨步全球的實驗。獨特的健保系統，加上完善的通訊基礎，官民充分合作，以及高技術...,https://technews.tw/2020/02/15/taiwan-mask-rea...,0,85,0,29988,"['3CExpert_tw', 'tech_tw']"
4,臺商第一手觀察：武漢疫情引爆中國 5 大經濟隱憂 | TechNews 科技新報,中國遭遇美國貿易制裁，現今又發生湖北武漢肺炎事件，將使中國出現以下重大經濟問題。筆者長期關注...,https://technews.tw/2020/02/15/wuhan-pneumonia...,0,85,0,29988,"['3CExpert_tw', 'tech_tw']"
...,...,...,...,...,...,...,...,...
495,「我不勇敢，誰幫你堅強！」新冠肺炎第十例痊癒訴心聲，感謝臺灣醫護 – 媽媽經｜專屬於媽媽的網站,如果要用一句話，來形容即將出院的心情，那我會這麼說：「我不勇敢誰幫你堅強！」中央流行疫情指揮...,https://mamaclub.com/learn/%e3%80%8c%e6%88%91%...,0,10,0,2520,"['Babyhome_tw', 'life_tw']"
496,民主黨總統初選 激進桑德斯竄出！溫和派憂 改挺彭博 | MoneyDJ新聞摘錄,許多溫和派的民主黨人對此憂心不已，轉向支持前紐約市長彭博,https://blog.moneydj.com/news/2020/02/14/%e6%b...,0,10,0,2520,None
497,【運動小姐】改善洋梨型身材：增加肌肉才能有效減脂｜女人迷 Womany,其實皮下脂肪與內臟脂肪相較之下，內臟脂肪比較容易消除。不論是皮下脂肪或內臟脂肪，都是身體為了...,https://womany.net/read/article/23033,0,10,0,2520,"['Fitness_tw', 'Sports_tw', 'life_tw']"
498,對抗新冠肺炎除戴口罩、勤洗手之外，外加提升免疫力4大穴位 – 媽媽經｜專屬於媽媽的網站,要抵抗肺炎除了戴口罩、勤洗手以外，我們也不能忽視照顧好自身的免疫力，今天就先來和大家分享能夠...,https://mamaclub.com/learn/%e5%b0%8d%e6%8a%97%...,0,10,0,2520,"['Babyhome_tw', 'life_tw']"


In [6]:
url = 'https://www.cupoy.com/MixNewsAction.do?op=getBucketGroups&nationid=TW&len=50'
response = requests.get(url)

group_map = {}
for group in response.json():
    group_map[group['groupid']] = group['name']

group_items = {}
for i in range(len(items)):
    item_groups = items.iloc[i]['groups']
    if item_groups:
        item_groups = item_groups.replace('[', '').replace(']', '').replace('\'', '').split(', ')
        for group in item_groups:
            group = group_map[group]
            if group not in group_items:
                group_items[group] = [i]
            else:
                group_items[group].append(i)

for groupid in sorted(group_items, key=lambda k: len(group_items[k]), reverse=True):
    print(groupid, len(group_items[groupid]))


商業 62
生活 58
科技 54
親子家庭 42
職場白領 38
享樂女性 37
3C達人 22
國際財經 21
運動 12
城市食旅 12
文青聚落 11
健身瘦身 10
御宅學園 9
設計 3


In [7]:

print('Trend News: ')
print('From keywordtrend', items.loc[items['keywordtrend'].idxmax(), ['title', 'keywordtrend', 'uidactscore', 'socialactscore', 'activescore']].to_dict())
print('From uidactscore', items.loc[items['uidactscore'].idxmax(), ['title', 'keywordtrend', 'uidactscore', 'socialactscore', 'activescore']].to_dict())
print('From socialactscore', items.loc[items['socialactscore'].idxmax(), ['title', 'keywordtrend', 'uidactscore', 'socialactscore', 'activescore']].to_dict())
print('From activescore', items.loc[items['activescore'].idxmax(), ['title', 'keywordtrend', 'uidactscore', 'socialactscore', 'activescore']].to_dict())


Trend News: 
From keywordtrend {'title': '疫情帶動宅經濟，電商徵才月薪上看 10 萬元 | TechNews 科技新報', 'keywordtrend': 0, 'uidactscore': 85, 'socialactscore': 0, 'activescore': 29988}
From uidactscore {'title': '疫情帶動宅經濟，電商徵才月薪上看 10 萬元 | TechNews 科技新報', 'keywordtrend': 0, 'uidactscore': 85, 'socialactscore': 0, 'activescore': 29988}
From socialactscore {'title': '重文輕武、重武輕文 - 專案與業務該重視誰的爭議 - 專案管理生活思維', 'keywordtrend': 0, 'uidactscore': 60, 'socialactscore': 394, 'activescore': 5925}
From activescore {'title': '疫情帶動宅經濟，電商徵才月薪上看 10 萬元 | TechNews 科技新報', 'keywordtrend': 0, 'uidactscore': 85, 'socialactscore': 0, 'activescore': 29988}
